## 任务一：HMM模型用于中文分词

任务一评分标准：
1. 共有8处TODO需要填写，每个TODO计1-2分，共9分，预计代码量30行；
2. 允许自行修改、编写代码完成，对于该情况，请补充注释以便于评分，否则结果不正确将导致较多的扣分；
3. 实验报告(python)/用于说明实验的文字块(jupyter notebook)不额外计分，但不写会导致扣分。

注：本任务仅在短句子上进行效果测试，因此对概率的计算可直接进行连乘。在实践中，常先对概率取对数，将连乘变为加法来计算，以避免出现数值溢出的情况。

导入HMM参数，初始化所需的起始概率矩阵，转移概率矩阵，发射概率矩阵

In [1]:
import pickle
import numpy as np

In [3]:
with open("hmm_parameters.pkl", "rb") as f:
    hmm_parameters = pickle.load(f)

# 非断字（B）为第0行，断字（I）为第1行
# 发射概率矩阵中，词典大小为65536，以汉字的ord作为行key
start_probability = hmm_parameters["start_prob"]  # shape(2,)
trans_matrix = hmm_parameters["trans_mat"]  # shape(2, 2)
emission_matrix = hmm_parameters["emission_mat"]  # shape(2, 65536)

In [4]:
print(start_probability)
print(trans_matrix)
print(emission_matrix[1][100])

[0.56262868 0.43737132]
[[0.18967134 0.81032866]
 [0.4821748  0.5178252 ]]
9.341673039710104e-06


定义待处理的句子

In [5]:
# TODO: 将input_sentence中的xxx替换为你的姓名（1分）
input_sentence = "马逸川是一名优秀的学生"


实现viterbi算法，并以此进行中文分词

In [6]:
def viterbi(sent_orig, start_prob, trans_mat, emission_mat):
    """
    viterbi算法进行中文分词

    Args:
        sent_orig: str - 输入的句子
        start_prob: numpy.ndarray - 起始概率矩阵
        trans_mat: numpy.ndarray - 转移概率矩阵
        emission_mat: numpy.ndarray - 发射概率矩阵

    Return:
        str - 中文分词的结果
    """
    
    #  将汉字转为数字表示
    sent_ord = [ord(x) for x in sent_orig]
    
    # `dp`用来储存不同位置每种标注（B/I）的最大概率值
    dp = np.zeros((2, len(sent_ord)), dtype=float)
    
    # `path`用来储存最大概率对应的上步B/I选择
    #  例如 path[1][7] == 1 意味着第8个（从1开始计数）字符标注I对应的最大概率，其前一步的隐状态为1（I）
    #  例如 path[0][5] == 1 意味着第6个字符标注B对应的最大概率，其前一步的隐状态为1（I）
    #  例如 path[1][1] == 0 意味着第2个字符标注I对应的最大概率，其前一步的隐状态为0（B）
    
    path = np.zeros((2, len(sent_ord)), dtype=int)
    
    #  TODO: 第一个位置的最大概率值计算（1分）
    dp[0][0], dp[1][0] = start_prob  
    dp[0][0] = dp[0][0] * emission_mat[0][sent_ord[0]]
    dp[1][0] = dp[1][0] * emission_mat[1][sent_ord[0]]
    
    # position 1 : 0 
    if (trans_mat[0][0] * dp[0][0] * emission_mat[0][sent_ord[0]] 
        > trans_mat[1][0] * dp[1][0] * emission_mat[0][sent_ord[0]]) :
        
        dp[0][1] = trans_mat[0][0] * dp[0][0] * emission_mat[0][sent_ord[0]]
        path[0][1] = 0  # 值表示前一位的状态(path is used for labeling)
        
    else :
        dp[0][1] = trans_mat[1][0] * dp[1][0] * emission_mat[0][sent_ord[0]]
        path[0][1] = 1
    
    # position 1 : 1
    if (trans_mat[0][1] * dp[0][0] * emission_mat[1][sent_ord[0]]
        > trans_mat[1][1] * dp[1][0] * emission_mat[1][sent_ord[0]]) :
        dp[1][1] = trans_mat[0][1] * dp[0][0] * emission_mat[1][sent_ord[0]]
        path[1][1] = 0
    else :
        dp[1][1] = trans_mat[1][1] * dp[1][0] * emission_mat[1][sent_ord[0]]
        path[1][1] = 1

    #  TODO: 其余位置的最大概率值计算（填充dp和path矩阵）（2分）
    
    length = len(sent_ord)
    for i in range(2, length):
        
        # position i : 0 
        if (dp[0][i-1] * trans_mat[0][0] * emission_mat[0][sent_ord[i]]
            > dp[1][i-1] * trans_mat[1][0] * emission_mat[0][sent_ord[i]]):
            
            dp[0][i] = dp[0][i-1] * trans_mat[0][0] * emission_mat[0][sent_ord[i]]
            path[0][i] = 0
            
        else :
            dp[0][i] = dp[1][i-1] * trans_mat[1][0] * emission_mat[0][sent_ord[i]]
            path[0][i] = 1
            
        # position i : 1    
        if (dp[0][i-1] * trans_mat[0][1] * emission_mat[1][sent_ord[i]]
            > dp[1][i-1] * trans_mat[1][1] * emission_mat[1][sent_ord[i]]) :
            
            dp[1][i] = dp[0][i-1] * trans_mat[0][1] * emission_mat[1][sent_ord[i]]
            path[1][i] = 0
            
        else :
            dp[1][i] = dp[1][i-1] * trans_mat[1][1] * emission_mat[1][sent_ord[i]]
            path[1][i] = 1
            
    
    #  `labels`用来储存每个位置最有可能的隐状态
    labels = [0 for _ in range(len(sent_ord))]
    
    #  TODO：计算labels每个位置上的值（填充labels矩阵）（1分）
    
    tmp_val = 0
    
    for i in reversed(range(length)):
        if i == length-1 : 
            if (dp[0][i]>dp[0][i]) : 
                labels[i] = 0 
                prev = path[0][i]
            else:
                labels[i] = 1
                prev = path[1][i]
        else : 
            labels[i] = prev
            prev = path[prev][i]
        
        
    #  根据lalels生成切分好的字符串
    sent_split = []
    for idx, label in enumerate(labels):
        if label == 1:
            sent_split += [sent_ord[idx], ord("/")]
        else:
            sent_split += [sent_ord[idx]]
    sent_split_str = "".join([chr(x) for x in sent_split])

    return sent_split_str
            

In [7]:
print("viterbi算法分词结果：", viterbi(input_sentence, start_probability, trans_matrix, emission_matrix))

viterbi算法分词结果： 马逸川/是/一名/优秀/的/学生/


实现前向算法，计算该句子的概率值

In [8]:
def compute_prob_by_forward(sent_orig, start_prob, trans_mat, emission_mat):
    """
    前向算法，计算输入中文句子的概率值

    Args:
        sent_orig: str - 输入的句子
        start_prob: numpy.ndarray - 起始概率矩阵
        trans_mat: numpy.ndarray - 转移概率矩阵
        emission_mat: numpy.ndarray - 发射概率矩阵

    Return:
        float - 概率值
    """
    
    #  将汉字转为数字表示
    sent_ord = [ord(x) for x in sent_orig]
    length = len(sent_ord)
    # `dp`用来储存不同位置每种隐状态（B/I）下，到该位置为止的句子的概率
    dp = np.zeros((2, len(sent_ord)), dtype=float)

    # TODO: 初始位置概率的计算（1分）
    dp[0][0], dp[1][0] = start_prob
    dp[0][0] = dp[0][0] * emission_mat[0][sent_ord[0]]
    dp[1][0] = dp[1][0] * emission_mat[1][sent_ord[0]]
        
    # TODO: 先计算其余位置的概率（填充dp矩阵），然后return概率值（1分）
    
    for i in range(1, length):
        dp[0][i] = dp[0][i-1] * trans_mat[0][0] * emission_mat[0][sent_ord[i]] + dp[1][i-1] * trans_mat[1][0] * emission_mat[0][sent_ord[i]]
        dp[1][i] = dp[0][i-1] * trans_mat[0][1] * emission_mat[1][sent_ord[i]] + dp[1][i-1] * trans_mat[1][1] * emission_mat[1][sent_ord[i]]
                   
    return sum([dp[i][len(sent_ord)-1] for i in range(2)])

实现后向算法，计算该句子的概率值

In [9]:
def compute_prob_by_backward(sent_orig, start_prob, trans_mat, emission_mat):
    """
    后向算法，计算输入中文句子的概率值

    Args:
        sent_orig: str - 输入的句子
        start_prob: numpy.ndarray - 起始概率矩阵
        trans_mat: numpy.ndarray - 转移概率矩阵
        emission_mat: numpy.ndarray - 发射概率矩阵

    Return:
        float - 概率值
    """
    
    #  将汉字转为数字表示
    sent_ord = [ord(x) for x in sent_orig]
    length = len(sent_ord)
    # `dp`用来储存不同位置每种隐状态（B/I）下，从结尾到该位置为止的句子的概率
    dp = np.zeros((2, len(sent_ord)), dtype=float)

    Row = trans_mat.shape[0]

    dp[:,(length-1):] = 1                 #最后的每一个元素赋值为1

    for t in reversed(range(length-1)):
        for n in range(Row):
            dp[n,t] = np.sum(dp[:,t+1]*trans_mat[n,:]*emission_mat[:,sent_ord[t+1]])

    
    # TODO: 先计算其余位置的概率（填充dp矩阵），然后return概率值（1分）
    pass

    return sum([dp[i][0] * start_prob[i] * emission_mat[i][sent_ord[0]] for i in range(2)])

In [10]:
print("前向算法概率：", compute_prob_by_forward(input_sentence, start_probability, trans_matrix, emission_matrix))
print("后向算法概率：", compute_prob_by_backward(input_sentence, start_probability, trans_matrix, emission_matrix))

前向算法概率： 4.039035028999559e-34
后向算法概率： 4.03903502899956e-34


## 任务二：BPE算法用于英文分词

任务二评分标准：

1. 共有7处TODO需要填写，每个TODO计1-2分，共9分，预计代码量50行；
2. 允许自行修改、编写代码完成，对于该情况，请补充注释以便于评分，否则结果不正确将导致较多的扣分；
3. 实验报告(python)/用于说明实验的文字块(jupyter notebook)不额外计分，但不写会导致扣分。

构建空格分词器，将语料中的句子以空格切分成单词，然后将单词拆分成字母加`</w>`的形式。例如`apple`将变为`a p p l e </w>`。

In [154]:
import re
import functools

In [156]:
_splitor_pattern = re.compile(r"[^a-zA-Z']+|(?=')")
_digit_pattern = re.compile(r"\d+")

def white_space_tokenize(corpus):
    """
    先正则化（字母转小写、数字转为N、除去标点符号），然后以空格分词语料中的句子，例如：
    输入 corpus=["I am happy.", "I have 10 apples!"]，
    得到 [["i", "am", "happy"], ["i", "have", "N", "apples"]]

    Args:
        corpus: List[str] - 待处理的语料

    Return:
        List[List[str]] - 二维List，内部的List由每个句子的单词str构成
    """

    tokeneds = [list(
        filter(lambda tkn: len(tkn)>0, _splitor_pattern.split(_digit_pattern.sub("N", stc.lower())))) for stc in corpus
    ]
    
    return tokeneds

#### 编写相应函数构建BPE算法需要用到的初始状态词典

In [157]:
def build_bpe_vocab(corpus):
    """
    将语料进行white_space_tokenize处理后，将单词每个字母以空格隔开、结尾加上</w>后，构建带频数的字典，例如：
    输入 corpus=["I am happy.", "I have 10 apples!"]，
    得到
    {
        'i </w>': 2,
        'a m </w>': 1,
        'h a p p y </w>': 1,
        'h a v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
     }

    Args:
        corpus: List[str] - 待处理的语料

    Return:
        Dict[str, int] - "单词分词状态->频数"的词典
    """

    tokenized_corpus = white_space_tokenize(corpus)
    # print(tokenized_corpus)
    bpe_vocab = dict() 
    
    # TODO: 完成函数体（1分）
    for tokens in tokenized_corpus : 
        # print(tokens)
        if isinstance(tokens, list):
            for token in tokens :
                token = list(token)
                for i in range(1,len(token)+int(len(token))//1,2):
                    token.insert(i, ' ')
                
                token.insert(len(token), '</w>')
                
                str = ''.join(token)
                    
                if str in bpe_vocab:
                    bpe_vocab[str] = bpe_vocab[str] + 1
                else:
                    bpe_vocab[str] = 1

    return bpe_vocab

#### test bpe_vocab

In [158]:
with open("data/news.2007.en.shuffled.deduped.train", encoding="utf-8") as f:
    training_corpus = list(map(lambda l: l.strip(), f.readlines()[:1000]))

print("Loaded training corpus.")

Loaded training corpus.


In [ ]:
training_bpe_vocab = build_bpe_vocab(training_corpus)

#### 编写所需的其他函数

##### get_bigram_freq

In [223]:
def get_bigram_freq(bpe_vocab):
    """
    统计"单词分词状态->频数"的词典中，各bigram的频次（假设该词典中，各个unigram以空格间隔），例如：
    输入 bpe_vocab=
    {
        'i </w>': 2,
        'a m </w>': 1,
        'h a p p y </w>': 1,
        'h a v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
    }
    得到
    {
        ('i', '</w>'): 2,
        ('a', 'm'): 1,
        ('m', '</w>'): 1,
        ('h', 'a'): 2,
        ('a', 'p'): 2,
        ('p', 'p'): 2,
        ('p', 'y'): 1,
        ('y', '</w>'): 1,
        ('a', 'v'): 1,
        ('v', 'e'): 1,
        ('e', '</w>'): 1,
        ('N', '</w>'): 1,
        ('p', 'l'): 1,
        ('l', 'e'): 1,
        ('e', 's'): 1,
        ('s', '</w>'): 1
    }

    Args:
        bpe_vocab: Dict[str, int] - "单词分词状态->频数"的词典

    Return:
        Dict[Tuple(str, str), int] - "bigram->频数"的词典
    """

    bigram_freq = dict()
    
    for token, value in bpe_vocab.items():
        
        token = token.split(' ')
        # print(token)
        
        for i in range(len(token)-1):
            gram = (token[i], token[i+1])
            if gram in bigram_freq:
                bigram_freq[gram] = bigram_freq[gram] + value
            else:
                bigram_freq[gram] = value
            
            
    return bigram_freq

##### Test

In [224]:
bigram_frequencies = get_bigram_freq(bpe_vocab=
    {
        'i </w>': 2,
        'a m </w>': 1,
        'h a p p y </w>': 1,
        'h a v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
    })

print(bigram_frequencies)

{('i', '</w>'): 2, ('a', 'm'): 1, ('m', '</w>'): 1, ('h', 'a'): 2, ('a', 'p'): 2, ('p', 'p'): 2, ('p', 'y'): 1, ('y', '</w>'): 1, ('a', 'v'): 1, ('v', 'e'): 1, ('e', '</w>'): 1, ('N', '</w>'): 1, ('p', 'l'): 1, ('l', 'e'): 1, ('e', 's'): 1, ('s', '</w>'): 1}


##### refresh

In [183]:
def refresh_bpe_vocab_by_merging_bigram(bigram, old_bpe_vocab):
    """
    在"单词分词状态->频数"的词典中，合并*指定的*bigram（即去掉对应的相邻unigram之间的空格），最后返回新的词典，例如：
    输入 bigram=('i', '</w>')，old_bpe_vocab=
    {
        'i </w>': 2,
        'a m </w>': 1,
        'h a p p y </w>': 1,
        'h a v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
    }
    得到
    {
        'i</w>': 2,
        'a m </w>': 1,
        'h a p p y </w>': 1,
        'h a v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
    }

    Args:
        old_bpe_vocab: Dict[str, int] - 初始"单词分词状态->频数"的词典

    Return:
        Dict[str, int] - 合并后的"单词分词状态->频数"的词典
    """
    
    new_bpe_vocab = dict()
    
    for token, value in old_bpe_vocab.items():
        token = token.split(' ')

        i=0
        while(True):
            # del the space if satisfying condiitons
            # print(token[i], token[i+1]) 
            if token[i]==bigram[0] and token[i+1]==bigram[1] :
                new_token = token[i] + token[i+1]
                token[i] = new_token
                del token[i+1]
            
            i = i + 1
            if i>=len(token)-1:
                break
                
        token = ' '.join(token)
        # print(token)
        new_bpe_vocab[token] = value
        
    return new_bpe_vocab

In [193]:
refresh_bpe_vocab_by_merging_bigram(bigram=('aa', 'i'),old_bpe_vocab=
    {
        'i </w>': 2,
        'a m i i </w> aa i </w>': 2,
        'a m </w>': 1,
        'h a p p y </w>': 1,
        'h a v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
    })

{'i </w>': 2,
 'a m i i </w> aai </w>': 2,
 'a m </w>': 1,
 'h a p p y </w>': 1,
 'h a v e </w>': 1,
 'N </w>': 1,
 'a p p l e s </w>': 1}

##### 由于下面多次使用字符串长度，这里封装一下。

In [161]:
def Count_Length(str):
    str = list(str)
    cnt = 0 
    i = 0 
    
    while(True):
        if str[i] != '<':
            cnt = cnt+1
            i = i+1
        else:
            if i+3 <= len(str) and ''.join(str[i:i+4])=='</w>':
                cnt = cnt+1
                i = i+4
            else:
                cnt = cnt+1
                i = i+1
        
        if i >= len(str):
            break

    return cnt

##### get_bpe_tokens

In [162]:
def get_bpe_tokens(bpe_vocab):
    """
    根据"单词分词状态->频数"的词典，返回所得到的BPE分词列表，并将该列表按照分词长度降序排序返回，例如：
    输入 bpe_vocab=
    {
        'i</w>': 2,
        'a m </w>': 1,
        'ha pp y </w>': 1,
        'ha v e </w>': 1,
        'N </w>': 1,
        'a pp l e s </w>': 1
    }
    得到
    [
        ('i</w>', 2),
        ('ha', 2),
        ('pp', 2),
        ('a', 2),
        ('m', 1),
        ('</w>', 5),
        ('y', 1),
        ('v', 1),
        ('e', 2),
        ('N', 1),
        ('l', 1),
        ('s', 1)
     ]

    Args:
        bpe_vocab: Dict[str, int] - "单词分词状态->频数"的词典

    Return:
        List[Tuple(str, int)] - BPE分词和对应频数组成的List
    """
    
    # TODO: 完成函数体（2分）
    # Reference: n-gram HW1
        
    tmp_vocab = {}
    for tokens, value in bpe_vocab.items():
        tokens = tokens.split(' ')
        
        for token in tokens:
            if token in tmp_vocab:
                tmp_vocab[token] = tmp_vocab[token] + value
            else:
                tmp_vocab[token] = value
            
    bpe_tokens = sorted(
                sorted(
                    map(lambda itm: (itm[0], itm[1]),  # 先按照item 1 排序
                        tmp_vocab.items()),
                    key=(lambda itm: itm[1]), reverse=True),
                key=(lambda itm: Count_Length(itm[0])), reverse=True) # 再按照第一位元素排序， 后面的值为频率
    # TODO: calculates the value of $N_r$
    # print(grams)
    return bpe_tokens

In [241]:
get_bpe_tokens(bpe_vocab=
    {
        'i</w>': 2,
        'a m </w>': 1,
        'ha ppy</w>': 1,
        'ha v e </w>': 1,
        'N </w>': 1,
        'a pp l e s </w>': 1
    })


[('ppy</w>', 1),
 ('i</w>', 2),
 ('ha', 2),
 ('pp', 1),
 ('</w>', 4),
 ('a', 2),
 ('e', 2),
 ('m', 1),
 ('v', 1),
 ('N', 1),
 ('l', 1),
 ('s', 1)]

##### 实现字符串分割函数

In [163]:
def Count_Length(str):

    if len(str)==0:
        return 0
    str = list(str)
    cnt = 0 
    i = 0 

    
    while(True):
        if str[i] != '<':
            cnt = cnt+1
            i = i+1
        else:
            if i+3 <= len(str) and ''.join(str[i:i+4])=='</w>':
                cnt = cnt+1
                i = i+4
            else:
                cnt = cnt+1
                i = i+1
        
        if i >= len(str):
            break

    return cnt

In [164]:
def Split_Token(str):  # 为了将最后的符号也区分出来
    str = list(str)

    i=0
    while(True):
        if str[i] != '<':
            i = i+1
        else:
            if i+3 <= len(str) and ''.join(str[i:i+4])=='</w>':
                str[i] = ''.join(str[i:i+4])
                del str[i+1:i+4]
                i = i + 1
            else:
                i = i+1
        
        if i >= len(str):
            break

    return str

In [62]:
str = "supermarket</w>"
Split_Token(str)

['s', 'u', 'p', 'e', 'r', 'm', 'a', 'r', 'k', 'e', 't', '</w>']

##### 对词表按长度排序

In [213]:
def Sort_length(bpe_vocab):
    sorted_list = sorted(
                        map(lambda itm: (itm[0], itm[1]),  # 先按照item 1 排序
                            bpe_vocab.items()),
                        key=(lambda itm: itm[0]), reverse=True)
    sorted_vocab = {}
    for itm in sorted_list:
        sorted_vocab[itm[0]] = itm[1]
    
    return sorted_vocab
            

In [214]:
Sort_length( bpe_vocab=
    { 
        'i</w>': 2,
        'a m </w>': 1,
        'ha ppp y </w>': 1,
        'ha v e </w>': 1,
        'N </w>': 1,
        'a pp l e s </w>': 1
    })

{'i</w>': 2,
 'ha v e </w>': 1,
 'ha ppp y </w>': 1,
 'a pp l e s </w>': 1,
 'a m </w>': 1,
 'N </w>': 1}

##### print tokenize

In [165]:
def print_bpe_tokenize(word, bpe_tokens):
    """
    根据按长度降序的BPE分词列表，将所给单词进行BPE分词，最后打印结果。
    
    思想是，对于一个待BPE分词的单词，按照长度顺序从列表中寻找BPE分词进行子串匹配，
    若成功匹配，则对该子串左右的剩余部分递归地进行下一轮匹配，直到剩余部分长度为0，
    或者剩余部分无法匹配（该部分整体由"<unknown>"代替）
    
    例1：
    输入 word="supermarket", bpe_tokens=[
        ("su", 20),
        ("are", 10),
        ("per", 30),
    ]
    最终打印 "su per <unknown>"

    例2：
    输入 word="shanghai", bpe_tokens=[
        ("hai", 1),
        ("sh", 1),
        ("an", 1),
        ("</w>", 1),
        ("g", 1)
    ]
    最终打印 "sh an g hai </w>"

    Args:
        word: str - 待分词的单词str
        bpe_tokens: List[Tuple(str, int)] - BPE分词和对应频数组成的List
    """
    
    # TODO: 请尝试使用递归函数定义该分词过程（2分）
     
    def bpe_tokenize(sub_word):
        if len(sub_word)==0:
            return ''
        len_word = Count_Length(sub_word)
        word_list = Split_Token(sub_word)
        
        flag = False
        
        for token, _ in bpe_tokens:  # 按长度查询词表
            len_token = Count_Length(token)
            token_list = Split_Token(token)
            for i in range(len_word): # 从左到右依次对比
                if i+len_token-1 >=len_word:
                    break
                # 相等时， 向两边递归
                
                if word_list[i:i+len_token] == token_list : 
                    flag = True            
                    left_word  = ''.join(word_list[0:i])
                    right_word = ''.join(word_list[i+len_token:len_word])
                    # print('left', left_word)
                    # print('right', right_word)
                    
                    # if len(left_word)
                    # print(token)
                    ret_str = bpe_tokenize(left_word) + token + ' ' + bpe_tokenize(right_word)
                    return ret_str
        
        if not flag:
            return '<unknown>'
                
    res = bpe_tokenize(word+"</w>")
    print(res)

In [225]:
print_bpe_tokenize(word="peoplearegood", bpe_tokens=[
        ("people", 1),
        ("are", 1),
        ("an", 1),
        ("</w>", 1),
        ("g", 1),
        ("shangh", 1)
    ])

people are g <unknown></w> 


##### 开始读取数据集并训练BPE分词器

In [166]:
with open("data/news.2007.en.shuffled.deduped.train", encoding="utf-8") as f:
    training_corpus = list(map(lambda l: l.strip(), f.readlines()[:1000]))

print("Loaded training corpus.")

Loaded training corpus.


In [246]:
training_iter_num = 300

training_bpe_vocab = build_bpe_vocab(training_corpus)
# 思路：合并最高频bigram
for i in range(training_iter_num):

    # TODO: 完成训练循环内的代码逻辑（2分）
    # if i == 10:
    #     print(training_bpe_vocab)
    #     break
    bigram_freq = get_bigram_freq(training_bpe_vocab)
    # print(bigram_freq)
    most_freq_gram = sorted(map(lambda itm: (itm[0], itm[1]),  # 先按照item 1 排序
                        bigram_freq.items()), key=(lambda itm: itm[1]), reverse=True)[0][0]
    # print(most_freq_gram)
    training_bpe_vocab = refresh_bpe_vocab_by_merging_bigram(most_freq_gram, training_bpe_vocab)
    training_bpe_vocab = Sort_length(training_bpe_vocab)
    
                
 
training_bpe_tokens = get_bpe_tokens(training_bpe_vocab)
print(training_bpe_tokens )   

[('people</w>', 47), ('ation</w>', 106), ('their</w>', 56), ('other</w>', 50), ('would</w>', 49), ('which</w>', 49), ('ement</w>', 44), ('after</w>', 44), ('about</w>', 41), ('that</w>', 198), ('with</w>', 155), ('said</w>', 153), ('from</w>', 99), ('tion</w>', 94), ('have</w>', 92), ('they</w>', 77), ('ting</w>', 75), ('ment</w>', 75), ('will</w>', 70), ('this</w>', 69), ('ight</w>', 59), ('ents</w>', 53), ('more</w>', 52), ('ence</w>', 50), ('ated</w>', 50), ('when</w>', 47), ('onal</w>', 46), ('year</w>', 45), ('were</w>', 45), ('inter', 44), ('also</w>', 44), ('ally</w>', 42), ('ates</w>', 42), ('ound</w>', 40), ('ould</w>', 34), ('ther</w>', 27), ('the</w>', 1218), ('ing</w>', 597), ('and</w>', 552), ('for</w>', 209), ('ers</w>', 178), ('ent</w>', 142), ('was</w>', 121), ('are</w>', 104), ('ted</w>', 101), ('his</w>', 100), ('ate</w>', 95), ('day</w>', 94), ('ons</w>', 87), ('ver</w>', 73), ('has</w>', 72), ('new</w>', 70), ('but</w>', 70), ('ere</w>', 68), ('all</w>', 68), ('one<

测试BPE分词器的分词效果

In [247]:
test_word = "naturallanguageprocessing"

print("naturallanguageprocessing 的分词结果为：")
print_bpe_tokenize(test_word, training_bpe_tokens)

naturallanguageprocessing 的分词结果为：
n a tur all an g u ag e pro c es s ing</w> 
